In [206]:
# Import modules
import requests
import csv
from bs4 import BeautifulSoup as bs
import os

# Helper methods
def convert_date(string):
    return string

def average(string):
    if "-" in string:
        string = string.split("-")
        avg = round((int(string[0]) + int(string[1]))/2)
        return str(avg)
    else:
        return string

def average_air(string):
    
    if "." in string:
        string = string.replace(".","")
    if "-" in string:
        string = string.split("-")
        avg = round((int(string[0]) + int(string[1]))/2)
        return str(avg)
    else:
        return string

def format_wind(string):
    result = string.split('m/s')
    
    if ',' in result[0]:
        result[0] = result[0].replace(',', '')
    if ',' in result[1]:
        result[1] = result[1].replace(',', '')
    
    speed = result[0].split()[0].strip()
    direction = result[1].strip()
    
    if "-" in speed:
        speed = average(speed)
    
    return [speed, direction]

# Test function(s) 2,0-4,5 m/s, Nordwest
format_wind("2,0-4,5 m/s, Nordwest")

['32', 'Nordwest']

In [199]:
# Track information that we want to scrape
aust_url = ["albert-park", "https://www.motorsport-total.com/formel-1/ergebnisse/", "", "/grosser-preis-von-australien/rennen"]
brit_url = ["silverstone", "https://www.motorsport-total.com/formel-1/ergebnisse/", "", "/grosser-preis-von-grossbritannien/rennen"]
span_url = ["catalunya", "https://www.motorsport-total.com/formel-1/ergebnisse/", "", "/grosser-preis-von-spanien/rennen"]
braz_url = ["interlagos", "https://www.motorsport-total.com/formel-1/ergebnisse/", "", "/grosser-preis-von-brasilien/rennen"]
belg_url = ["de-spa", "https://www.motorsport-total.com/formel-1/ergebnisse/", "", "/grosser-preis-von-belgien/rennen"]

dates = ['2012', '2013', '2014', '2015'] 
location_urls = [aust_url, brit_url, span_url, braz_url, belg_url]

In [209]:
def scrape_weather():
    # Create empty csv for writing
    file = open('data/track_weather.csv', 'w', newline='')
    writer = csv.writer(file)
    writer.writerow(['track', 'date', 'local_time', 'weather', 'temp', 'track_temp', 'humidity', 'air_pressure', 'wind_speed', 'wind_direction'])
    
    for raw_url in location_urls:
        track = raw_url[0]
        print("INFO: NOW SCRAPING " + raw_url[0])
        
        for year in dates:
            # Create response & find raw weather data
            raw_url[2] = year
            url = raw_url[1] + raw_url[2] + raw_url[3]       
            response = requests.get(url)
            soup = bs(response.content, 'html.parser')
            raw_weather = soup.find('div', {'id': 'session-info'}).find_all('td')

            # Assign raw to actual variables
            date = raw_weather[1].text.strip()
            local_time = raw_weather[5].text.strip() + ":00"
            weather = raw_weather[13].text.strip()
            temp = raw_weather[15].text.strip()[:-2]
            track_temp = raw_weather[21].text.strip()[:-2]
            humidity = raw_weather[17].text.strip()[:-1]
            air_pressure = raw_weather[19].text.strip()[:-5]
            wind = raw_weather[23].text.strip()

            # Format particular data
            temp = average(temp)
            track_temp = average(track_temp)
            humidity = average(humidity)
            air_pressure = average_air(air_pressure)
            wind_info = format_wind(wind)
            wind_speed = wind_info[0]
            wind_direction = wind_info[1]

            # Write to file
            writer.writerow([track, date, local_time, weather, temp, track_temp, humidity, air_pressure, wind_speed, wind_direction])
            print("LOG: " + year + " FINISHED")
    
    print("INFO: COMPLETED SCRAPING WEB DATA")
    file.close()

In [210]:
scrape_weather()

INFO: NOW SCRAPING albert-park
LOG: 2012 FINISHED
LOG: 2013 FINISHED
LOG: 2014 FINISHED
LOG: 2015 FINISHED
INFO: NOW SCRAPING silverstone
LOG: 2012 FINISHED
LOG: 2013 FINISHED
LOG: 2014 FINISHED
LOG: 2015 FINISHED
INFO: NOW SCRAPING catalunya
LOG: 2012 FINISHED
LOG: 2013 FINISHED
LOG: 2014 FINISHED
LOG: 2015 FINISHED
INFO: NOW SCRAPING interlagos
LOG: 2012 FINISHED
LOG: 2013 FINISHED
LOG: 2014 FINISHED
LOG: 2015 FINISHED
INFO: NOW SCRAPING de-spa
LOG: 2012 FINISHED
LOG: 2013 FINISHED
LOG: 2014 FINISHED
LOG: 2015 FINISHED
INFO: COMPLETED SCRAPING WEB DATA
